In [1]:
from StreetGraph import StreetGraph
from Graphworld import GraphEnv

import sys
sys.path.insert(0,"")
from gym_example.envs.Gridworld_Gym import Gridworld_v1


#graph_meinheim=StreetGraph('meinheim')
#graph_meinheim_trips = graph_meinheim.trips

In [2]:
env=GraphEnv()

In [31]:
env.availableActions()

[[{'type': 'wait'}], [{'type': 'ownRide'}]]

In [72]:
env.step(action=2)

[[{'type': 'wait'}], [{'type': 'ownRide'}]]


(5, -1, False, {})

In [3]:
env.graph.get_nodeid_by_index(env.position)

290333444

In [4]:
graph_meinheim=StreetGraph('meinheim')
graph_meinheim_trips = StreetGraph('meinheim').trips
print(graph_meinheim_trips)

      tripid  pickup_node  dropoff_node  pickup_day  pickup_hour  \
0          0    307664242     290333858           1           23   
1          1    280324485     306225115           1            1   
2          2    306221807     291556214           1           20   
3          3    306221765     307662889           1            2   
4          4    309441562     354863048           1           17   
...      ...          ...           ...         ...          ...   
3995    3995    306225115     309450366           1            3   
3996    3996    307662889     306219024           1           10   
3997    3997    280324491     290333400           1           17   
3998    3998    309450340     306219024           1            7   
3999    3999    307661801     309450229           1           12   

      pickup_minute     pickup_datetime  \
0                18 2022-01-01 23:18:00   
1                42 2022-01-01 01:42:00   
2                10 2022-01-01 20:10:00   
3          

In [2]:
#without ppo trainer
def run_one_episode (env):
    env.reset()
    sum_reward = 0
    for i in range(30):
        print(env.action_space)
        action = env.action_space.sample()
        state, reward, done, info = env.step(action)
        sum_reward+=reward
        #env.render()
        if done:
            print("sum_reward: ",sum_reward, " time: ",env.time,  "deadline time: ", env.deadline,"pickup time: ", env.pickup_time )
            break

        print("sum_reward: ",sum_reward, " time: ",env.time, "deadline time: ", env.deadline, "pickup time: ", env.pickup_time)
    return sum_reward

env=GraphEnv()
for i in range(1):
    run_one_episode (env)

Discrete(4)
[[{'type': 'wait'}], [{'type': 'ownRide'}], {'departure_time': datetime.datetime(2022, 1, 1, 12, 48, 40), 'target_node': 306221900, 'route': [290333444, 304525639, 307663269, 307662889, 307662284, 309450366, 307662242, 306222128, 306222385, 306221900]}, {'departure_time': datetime.datetime(2022, 1, 1, 12, 47, 2), 'target_node': 307664334, 'route': [290333444, 290333443, 307664257, 307664242, 307664334]}]
action ==  2  New Position 19
sum_reward:  -1  time:  2022-01-01 12:53:15.400000 deadline time:  2022-01-01 15:45:00 pickup time:  2022-01-01 12:45:00
Discrete(7)
[[{'type': 'wait'}], [{'type': 'ownRide'}], {'departure_time': datetime.datetime(2022, 1, 1, 12, 55, 9), 'target_node': 306221960, 'route': [306221900, 306221960]}, {'departure_time': datetime.datetime(2022, 1, 1, 12, 55, 56), 'target_node': 309441547, 'route': [306221900, 306221960, 306222676, 306221941, 399384843, 306221950, 309441547]}, {'departure_time': datetime.datetime(2022, 1, 1, 12, 57, 22), 'target_node'

In [6]:
import numpy as np
import pandas as pd
import json
import os
import shutil
import sys
import gym

import ray
from ray.rllib.agents.ppo import PPOTrainer, DEFAULT_CONFIG

In [8]:
ray.init()

RuntimeError: Maybe you called ray.init twice by accident? This error can be suppressed by passing in 'ignore_reinit_error=True' or by calling 'ray.shutdown()' prior to 'ray.init()'.

In [9]:
trainer_config = DEFAULT_CONFIG.copy()
trainer_config['num_workers'] = 1
trainer_config["train_batch_size"] = 400
trainer_config["sgd_minibatch_size"] = 64
trainer_config["num_sgd_iter"] = 10
trainer_config["framework"] = "torch"

In [19]:
trainer = PPOTrainer(trainer_config,GraphEnv )

(RolloutWorker pid=36475) 2022-04-02 09:54:22,481	WARNING deprecation.py:45 -- DeprecationWarning: `rllib.env.remote_vector_env.RemoteVectorEnv` has been deprecated. Use `ray.rllib.env.remote_base_env.RemoteBaseEnv` instead. This will raise an error in the future!
2022-04-02 09:54:23,311	WARNING util.py:55 -- Install gputil for GPU system monitoring.


In [11]:
checkpoint_root = "tmp/ppo/gridworld"
shutil.rmtree(checkpoint_root, ignore_errors=True, onerror=None)   # clean up old runs

In [20]:
trainer.train()

(RolloutWorker pid=36475) [[{'type': 'wait'}], [{'type': 'ownRide'}]]
(RolloutWorker pid=36475) action == wait 
(RolloutWorker pid=36475) [[{'type': 'wait'}], [{'type': 'ownRide'}], {'departure_time': datetime.datetime(2022, 1, 1, 11, 56, 36), 'target_node': 307664242, 'route': [290333444, 290333443, 307664257, 307664242]}, {'departure_time': datetime.datetime(2022, 1, 1, 11, 56, 24), 'target_node': 307664257, 'route': [290333444, 290333443, 307664257]}, {'departure_time': datetime.datetime(2022, 1, 1, 11, 57, 42), 'target_node': 306222383, 'route': [290333444, 304525639, 307663269, 307662889, 307662284, 309450366, 307662242, 306222128, 306222385, 306222383]}, {'departure_time': datetime.datetime(2022, 1, 1, 11, 57, 34), 'target_node': 307664257, 'route': [290333444, 290333443, 307664257]}, {'departure_time': datetime.datetime(2022, 1, 1, 11, 56, 52), 'target_node': 7756059836, 'route': [290333444, 304525639, 307663269, 307662889, 307662284, 307661801, 307662149, 7756059845, 7756059836

{'episode_reward_max': 100.0,
 'episode_reward_min': 50.0,
 'episode_reward_mean': 94.17857142857143,
 'episode_len_mean': 6.821428571428571,
 'episode_media': {},
 'episodes_this_iter': 56,
 'policy_reward_min': {},
 'policy_reward_max': {},
 'policy_reward_mean': {},
 'custom_metrics': {},
 'hist_stats': {'episode_reward': [98.0,
   100.0,
   95.0,
   88.0,
   92.0,
   89.0,
   98.0,
   99.0,
   92.0,
   97.0,
   86.0,
   85.0,
   100.0,
   96.0,
   89.0,
   99.0,
   97.0,
   91.0,
   88.0,
   98.0,
   97.0,
   99.0,
   92.0,
   93.0,
   91.0,
   93.0,
   97.0,
   91.0,
   50.0,
   99.0,
   99.0,
   96.0,
   100.0,
   94.0,
   99.0,
   94.0,
   98.0,
   93.0,
   97.0,
   92.0,
   91.0,
   94.0,
   98.0,
   100.0,
   90.0,
   94.0,
   98.0,
   94.0,
   99.0,
   99.0,
   99.0,
   92.0,
   99.0,
   87.0,
   99.0,
   100.0],
  'episode_lengths': [3,
   1,
   6,
   13,
   9,
   12,
   3,
   2,
   9,
   4,
   15,
   16,
   1,
   5,
   12,
   2,
   4,
   10,
   13,
   3,
   4,
   2,
   9,
 

NameError: name 'result' is not defined

In [16]:
results = []
episode_data = []
episode_json = []
n_iter=5

for n in range(n_iter):
    result = trainer.train()
    results.append(result)
    
    episode = {'n': n, 
               'episode_reward_min': result['episode_reward_min'], 
               'episode_reward_mean': result['episode_reward_mean'], 
               'episode_reward_max': result['episode_reward_max'],  
               'episode_len_mean': result['episode_len_mean']
              }
    
    episode_data.append(episode)
    episode_json.append(json.dumps(episode))
    file_name = trainer.save(checkpoint_root)
    
    print(f'{n+1:3d}: Min/Mean/Max reward: {result["episode_reward_min"]:8.4f}/{result["episode_reward_mean"]:8.4f}/{result["episode_reward_max"]:8.4f}, len mean: {result["episode_len_mean"]:8.4f}. Checkpoint saved to {file_name}')

StopIteration: 

In [ ]:
ray.shutdown()